In [1]:
import numpy as np
from time import sleep
from neurons import poisson, LIF

from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import normalize

from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.palettes import mpl

output_notebook()

Loading BokehJS ...

In [2]:
class spiking_neural_network:
    def __init__(self):
        self.x = []
        self.y = []
        self.layer0 = []
        self.layer1 = []
        self.layer2 = []
        self.layer0_eval = []
        self.w01 = []
        self.w12 = []

    def RL_Curve(self, t):
        A_plus = 0.8
        A_minus = 0.3
        tau_plus = 8
        tau_minus = 5

        if t>0:
            return A_plus*np.exp(-float(t)/tau_plus)
        else:
            return A_minus*np.exp(float(t)/tau_minus)

    def update_weights(self, w, delta_w):
        scale = 1
        w_min = -1.2*scale
        w_max = 1.5*scale
        sigma = 0.1
        if delta_w < 0:
            return w + sigma*delta_w*(w - abs(w_min))*scale
        else:
            return w + sigma*delta_w*(w_max - w)*scale

    def generate_network(self):
        for i in range(30):
            self.layer0.append(poisson())
        for i in range(100):
            self.layer1.append(LIF())
        for i in range(2):
            self.layer2.append(LIF())
            self.layer2[i].update_threshold(10)

        self.w01 = np.ones((30, 100))
        self.w12 = np.random.rand(100, 2)

    def reset_network(self):
        for neuron in self.layer1:
            neuron.reset_neuron()

        for neuron in self.layer2:
            neuron.reset_neuron()

        self.layer0_eval = []


    def predict(self, x, time):
        self.reset_network()
        for i in range(len(self.layer0)):
            self.layer0_eval.append(self.layer0[i].execute(x[i], time))

        syn01 = []
        for i in range(len(self.layer1)):
            summ = 0
            for j in range(len(self.layer0)):
                summ += self.layer0_eval[j]*self.w01[j, i]
            syn01.append(summ)
        syn01 = np.array(syn01)

        for i, t in enumerate(time):
            for j in range(len(self.layer1)):
                self.layer1[j].execute(syn01[j, i], t)

            for j in range(len(self.layer2)):
                syn = 0
                for k in range(len(self.layer1)):
                    syn += self.w12[k, j]*self.layer1[k].out
                self.layer2[j].execute(syn, t)
        
        return np.argmax([self.layer2[0].mem_pot, self.layer2[1].mem_pot])

        
    def train(self, x, y, time, epochs):
        t_back = -10
        t_forwd = 10
        t_post = 0
        for epoch in range(epochs):
            for row in range(np.shape(x)[0]):
                
                self.reset_network()
                print('{:.2f} %'.format(100*(row+1)/np.shape(x)[0]))

                for i in range(len(self.layer0)):
                    self.layer0_eval.append(self.layer0[i].execute(x[row, i], time))

                syn01 = []
                for i in range(len(self.layer1)):
                    sum = 0
                    for j in range(len(self.layer0)):
                        sum += self.layer0_eval[j]*self.w01[j, i]
                    syn01.append(sum)
                syn01 = np.array(syn01)

                for i, t in enumerate(time):
                    for j in range(len(self.layer1)):
                        self.layer1[j].execute(syn01[j, i], t)
                        # if self.layer1[j].out == 1:
                        #     t_post = t
                        # if t_post:
                        #     self.w01[j, i] = self.update_weights(self.w01[j, i], self.RL_Curve(t-t_post))


                    for j in range(len(self.layer2)):
                        syn = 0
                        for k in range(len(self.layer1)):
                            syn += self.w12[k, j]*self.layer1[k].out
                        self.layer2[j].execute(syn, t)
                    
                    for j in range(len(self.layer2)):
                        if self.layer2[j].out == 1:
                                t_post = t
                        for k in range(len(self.layer1)):
                            if t_post:
                                self.w12[k, j] = self.update_weights(self.w12[k, j], self.RL_Curve(t-t_post))

In [3]:
T = 100
dt = 0.1
time = np.arange(0, T+dt, dt)

dataset = load_breast_cancer()
x = dataset.data
label = dataset.target

x = normalize(x)
new_x = x[0].reshape(30, 1)
y = np.zeros((np.size(label, 0), len(np.unique(label))))
for i in range(np.size(label, 0)):
    y[i, label[i]] = 1

In [4]:
snn = spiking_neural_network()
snn.generate_network()
snn.train(x, y, time, epochs=1)

0.18 %
0.35 %
0.53 %
0.70 %
0.88 %
1.05 %
1.23 %
1.41 %
1.58 %
1.76 %
1.93 %
2.11 %
2.28 %
2.46 %
2.64 %
2.81 %
2.99 %
3.16 %
3.34 %
3.51 %
3.69 %
3.87 %
4.04 %
4.22 %
4.39 %
4.57 %
4.75 %
4.92 %
5.10 %
5.27 %
5.45 %
5.62 %
5.80 %
5.98 %
6.15 %
6.33 %
6.50 %
6.68 %
6.85 %
7.03 %
7.21 %
7.38 %
7.56 %
7.73 %
7.91 %
8.08 %
8.26 %
8.44 %
8.61 %
8.79 %
8.96 %
9.14 %
9.31 %
9.49 %
9.67 %
9.84 %
10.02 %
10.19 %
10.37 %
10.54 %
10.72 %
10.90 %
11.07 %
11.25 %
11.42 %
11.60 %
11.78 %
11.95 %
12.13 %
12.30 %
12.48 %
12.65 %
12.83 %
13.01 %
13.18 %
13.36 %
13.53 %
13.71 %
13.88 %
14.06 %
14.24 %
14.41 %
14.59 %
14.76 %
14.94 %
15.11 %
15.29 %
15.47 %
15.64 %
15.82 %
15.99 %
16.17 %
16.34 %
16.52 %
16.70 %
16.87 %
17.05 %
17.22 %
17.40 %
17.57 %
17.75 %
17.93 %
18.10 %
18.28 %
18.45 %
18.63 %
18.80 %
18.98 %
19.16 %
19.33 %
19.51 %
19.68 %
19.86 %
20.04 %
20.21 %
20.39 %
20.56 %
20.74 %
20.91 %
21.09 %
21.27 %
21.44 %
21.62 %
21.79 %
21.97 %
22.14 %
22.32 %
22.50 %
22.67 %
22.85 %
23.02 %
23.20 %


In [5]:
y_pred = []
for i in range(np.shape(x)[0]):
    print(i)
    y_pred.append(snn.predict(x[i], time))
 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [6]:
p = figure(height=400)
color = mpl['Plasma'][3]
for i in range(len(snn.layer2)):
    # p.vbar(x=time, top=snn.layer2[i].out_array, color=color[i], legend_label='LIF {}'.format(i+1))
    p.line(x=time, y=snn.layer2[i].pot_plot, color=color[i], legend_label='LIF {}'.format(i+1), line_width=1.5)
    # p.vbar(x=time, top=snn.layer1[i].pot_plot, legend_label='LIF {}'.format(i+1), line_width=1.5)
show(p)

In [7]:
y_pred

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
